In [1]:
local_model = "/Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_0.gguf"
# Load the local LLM model
#llama_model = Llama(model_path=local_model, n_ctx=56000, verbose=False)
from llama_index.llms.llama_cpp import LlamaCPP
# Initialize LLM properly using LlamaCPP wrapper
llm = LlamaCPP(
    model_path=local_model,
    model_kwargs={
        "n_ctx": 2048,
        "n_threads": 4,
    }
)

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
llama_model_loader: loaded meta data with 26 key-value pairs and 579 tensors from /Users/maxhuang/projects/llm/models/Qwen/Qwen2.5-Coder-14B-Instru

In [2]:
# pip install llama-index-embeddings-huggingface
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_path = "/Users/maxhuang/projects/llm/models/all-MiniLM-L6-v2"
embed_model = HuggingFaceEmbedding(model_name=embedding_model_path)

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

In [8]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
#from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        TitleExtractor(),
    ]
)

data_path = "/Users/maxhuang/projects/llm/langchain-book-examples/data_small"
documents = SimpleDirectoryReader(data_path).load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
response = query_engine.query("List a few sports")

# Access source nodes from response
source_nodes = response.source_nodes

# Print response and source documents
print("=== Query Response ===")
print(response)

print("\n=== Source Documents ===")
for i, node in enumerate(source_nodes):
    print(f"\n=== Document {i+1} ===")
    print(f"Score: {node.score if hasattr(node, 'score') else 'N/A'}")
    print(f"Content: {node.text}")
    print(f"Metadata: {node.metadata}")

Llama.generate: 166 prefix-match hit, remaining 1 prompt tokens to eval

llama_print_timings:        load time =    8728.20 ms
llama_print_timings:      sample time =       1.01 ms /    20 runs   (    0.05 ms per token, 19723.87 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (     nan ms per token,      nan tokens per second)
llama_print_timings:        eval time =    2710.50 ms /    20 runs   (  135.53 ms per token,     7.38 tokens per second)
llama_print_timings:       total time =    2719.65 ms /    20 tokens


=== Query Response ===
1. Football
2. Basketball
3. Tennis
4. Swimming
5. Hockey

=== Source Documents ===

=== Document 1 ===
Score: 0.41206990037799474
Content: Sport is generally recognised as activities based in physical athleticism or physical dexterity.[3] Sports are usually governed by rules to ensure fair competition and consistent adjudication of the winner.

"Sport" comes from the Old French desport meaning "leisure", with the oldest definition in English from around 1300 being "anything humans find amusing or entertaining".[4]

Other bodies advocate widening the definition of sport to include all physical activity and exercise. For instance, the Council of Europe include all forms of physical exercise, including those completed just for fun.
Metadata: {'file_path': '/Users/maxhuang/projects/llm/langchain-book-examples/data_small/sports.txt', 'file_name': 'sports.txt', 'file_type': 'text/plain', 'file_size': 601, 'creation_date': '2024-12-14', 'last_modified_date': '2024-12-1

: 